In [1]:
import sys
sys.path.append('../')
from src.models.FeedbackModelBuilder import VGG16FeedbackFrozen4BlockTo4Block,\
                                            VGG16FeedbackFrozen5BlockTo5Block,\
                                            VGG16FeedbackFrozen4BlockTo1Block,\
                                            VGG16FeedbackFrozen5BlockTo3Block,\
                                            VGG16FeedbackFrozen5BlockTo4Block

from src.models.ForwardModelBuilder import VGG16CustomFrozen
from src.Evaluation import run_mcnemar_test
from src.DatasetNoise import tf_gaussian_noise, tf_salt_pepper_noise
import tensorflow as tf
import itertools
import os

2023-02-02 22:13:16.199728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 22:13:23.259071: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-02 22:13:40.742851: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-02-02 22:13:40.755332: W tensorflow/compil

In [2]:
path_to_datasets = '../../../experiments/create_datasets/normalize_dataset/saved_datasets/'
test_ds_prepared_without_batch = tf.data.Dataset.load(os.path.join(path_to_datasets, "test_ds_without_batching"))

2023-02-02 22:14:16.851433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-02-02 22:14:16.851483: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-02 22:14:16.851510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uc2n368.localdomain): /proc/driver/nvidia/version does not exist
2023-02-02 22:14:16.905963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate co

In [3]:
batch_size = 16
num_classes = 16
img_size = 224
input_shape = (None, img_size, img_size, 3)
test_ds = test_ds_prepared_without_batch.batch(batch_size).prefetch(4)
test_ds_gaussian_noise = test_ds_prepared_without_batch.map(tf_gaussian_noise).batch(batch_size).prefetch(4)
test_ds_salt_pepper_noise = test_ds_prepared_without_batch.map(tf_salt_pepper_noise).batch(batch_size).prefetch(4)

In [4]:
name_model_path = {
"frozenVGG16-Feedback_5block_to_3block": {
    "model_class": VGG16FeedbackFrozen5BlockTo3Block,
    "checkpoint_filepath":"../models/frozenVGG16/Feedback_5block_to_3block/checkpoint"
                                         },
"frozenVGG16-Feedback_4block_to_4block": {
    "model_class": VGG16FeedbackFrozen4BlockTo4Block,
    "checkpoint_filepath":"../models/frozenVGG16/Feedback_4block_to_4block/checkpoint"
                                         },
"frozenVGG16-Feedback_4block_to_1block": {
    "model_class": VGG16FeedbackFrozen4BlockTo1Block,
    "checkpoint_filepath":"../models/frozenVGG16/Feedback_4block_to_1block/checkpoint"
                                         },
"frozenVGG16-Feedback_5block_to_4block": {
    "model_class": VGG16FeedbackFrozen5BlockTo4Block,
    "checkpoint_filepath":"../models/frozenVGG16/Feedback_5block_to_4block/checkpoint"
                                         },
"frozenVGG16-Feedback_5block_to_5block": {
    "model_class": VGG16FeedbackFrozen5BlockTo5Block,
    "checkpoint_filepath":"../models/frozenVGG16/Feedback_5block_to_5block/checkpoint"
                                         },
"frozenVGG16-Forward": {
    "model_class": VGG16CustomFrozen,
    "checkpoint_filepath":"../models/frozenVGG16/Forward/checkpoint"
                                         }
}

In [5]:
def build_model(model_class, model_checkpoint_filepath):
    sample = next(iter(test_ds))[0]
    model = model_class()
    model.build(input_shape)
    model(sample)
    model.summary(show_trainable=True)
    model.load_weights(model_checkpoint_filepath)
    return model

In [ ]:
for model_1, model_2 in itertools.combinations([(k,v) for k,v in name_model_path.items()], 2):
    print(model_1)
    print(model_2)
    model_1_class = model_1[1]["model_class"]
    model_1_checkpoint_filepath = model_1[1]["checkpoint_filepath"]
    model_1_name = model_1[0]
    
    model_2_class = model_2[1]["model_class"]
    model_2_checkpoint_filepath = model_2[1]["checkpoint_filepath"]
    model_2_name = model_2[0]
    
    model_1 = build_model(model_1_class, model_1_checkpoint_filepath)
    model_2 = build_model(model_2_class, model_2_checkpoint_filepath)
    
    print(run_mcnemar_test(model_1, model_2, test_ds))
    break

('frozenVGG16-Feedback_5block_to_3block', {'model_class': <class 'src.models.FeedbackModelBuilder.VGG16FeedbackFrozen5BlockTo3Block'>, 'checkpoint_filepath': '../models/frozenVGG16/Feedback_5block_to_3block/checkpoint'})
('frozenVGG16-Feedback_4block_to_4block', {'model_class': <class 'src.models.FeedbackModelBuilder.VGG16FeedbackFrozen4BlockTo4Block'>, 'checkpoint_filepath': '../models/frozenVGG16/Feedback_4block_to_4block/checkpoint'})
Model: "FFmodel"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 model (Functional)          (None, None, None, 128)   260160    N          
                                                                            
 conv2d (Conv2D)             multiple                  295168    Y          
                                                                            
 conv2d_1 (Conv2D)           multiple                  590080    Y          
  

In [ ]:
sample = next(iter(test_ds))[0]
model = model_class()
model.build(input_shape)
model(sample)
model.summary(show_trainable=True)
model.load_weights(checkpoint_filepath)